# Notebook 1: Function Calling

Give the model a set of tools — it decides which one to call, with what arguments, based on the question. No hardcoded routing.

## Setup

In [ ]:
import json
import sys
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI

sys.path.insert(0, str(Path("..").resolve()))
from tools import TOOL_SCHEMAS, TOOL_FUNCTIONS

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

## Part 1: Inspect the tool schemas

In [ ]:
# The model reads these schemas to decide when and how to call each tool.
for schema in TOOL_SCHEMAS:
    print(json.dumps(schema, indent=2))
    print()

## Part 2: Send a question — get a tool call back

In [ ]:
question = "What does Agentic AI Liability Insurance cover?"

messages = [
    {
        "role": "system",
        "content": (
            "You are a helpful assistant for AI Agent Insure. "
            "Use the available tools to answer questions accurately. "
            "Do not guess — if a tool can answer the question, use it."
        )
    },
    {"role": "user", "content": question}
]

# tool_choice="auto" — the model decides whether to call a tool or answer directly.
response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=TOOL_SCHEMAS,
    tool_choice="auto",
    temperature=0
)

print("Stop reason:", response.choices[0].finish_reason)
print()
print("Full response message:")
print(response.choices[0].message)

## Part 3: Inspect the tool call

In [ ]:
# tool_calls contains: id, function.name, function.arguments (JSON string)
tool_call = response.choices[0].message.tool_calls[0]

print("Tool the model wants to call:", tool_call.function.name)
print("Arguments (raw JSON string):  ", tool_call.function.arguments)
print()

args = json.loads(tool_call.function.arguments)
print("Arguments (parsed dict):      ", args)

## Part 4: Execute the tool, feed the result back, get the final answer

In [ ]:
fn = TOOL_FUNCTIONS[tool_call.function.name]
tool_result = fn(**args)
print("Tool returned:", tool_result)
print()

# Append the model's tool_call message, then the tool result with role="tool".
# tool_call_id links this result back to the specific call the model made.
messages.append(response.choices[0].message)
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": tool_result
})

# Second API call — model reads the result and writes the final answer.
final_response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=TOOL_SCHEMAS,
    temperature=0
)

print("Stop reason:", final_response.choices[0].finish_reason)
print()
print("Final answer:")
print(final_response.choices[0].message.content)

## Part 5: Three questions — three different tools

In [ ]:
def single_tool_call_demo(question: str) -> None:
    msgs = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant for AI Agent Insure. "
                "Use the available tools to answer questions accurately."
            )
        },
        {"role": "user", "content": question}
    ]

    r1 = client.chat.completions.create(
        model=MODEL, messages=msgs, tools=TOOL_SCHEMAS,
        tool_choice="auto", temperature=0
    )

    tc = r1.choices[0].message.tool_calls[0]
    args = json.loads(tc.function.arguments)
    result = TOOL_FUNCTIONS[tc.function.name](**args)

    print(f"Q: {question}")
    print(f"   → Tool called : {tc.function.name}")
    print(f"   → Arguments   : {args}")
    print(f"   → Tool result : {result}")

    msgs.append(r1.choices[0].message)
    msgs.append({"role": "tool", "tool_call_id": tc.id, "content": result})
    r2 = client.chat.completions.create(
        model=MODEL, messages=msgs, tools=TOOL_SCHEMAS, temperature=0
    )
    print(f"   → Final answer : {r2.choices[0].message.content}")
    print()


single_tool_call_demo("What does Model & Data Security Insurance cover?")
single_tool_call_demo("How much would Agentic AI Liability Insurance cost for a startup?")
single_tool_call_demo("Can a healthcare company get coverage from AI Agent Insure?")

## Part 6: No tool needed — model answers directly

In [ ]:
general_question = "What is AI Agent Insure?"

msgs = [
    {
        "role": "system",
        "content": (
            "You are a helpful assistant for AI Agent Insure, a specialty insurer "
            "for AI systems, autonomous agents, and ML infrastructure. "
            "Use the available tools to answer questions accurately."
        )
    },
    {"role": "user", "content": general_question}
]

r = client.chat.completions.create(
    model=MODEL, messages=msgs, tools=TOOL_SCHEMAS,
    tool_choice="auto", temperature=0
)

# finish_reason will be "stop" (not "tool_calls") — no tool needed.
print("Stop reason:", r.choices[0].finish_reason)
print("Answer:", r.choices[0].message.content)

## Key concepts

- `finish_reason: "tool_calls"` → model is mid-task, waiting for a result
- `finish_reason: "stop"` → model is done
- The model never runs Python — it emits a JSON instruction, we execute it
- **Next:** Notebook 2 wraps this into an automated loop